# Домашняя работа 3. Гибридные рекомендательные системы

Датасет ml-latest
 - Вспомнить подходы, которые мы разбирали
 - Выбрать понравившийся подход к гибридным системам
 - Написать свою

Решением будет ссылка на гитхаб с готовым ноутбуком

## Импорт библиотек

In [26]:
import pandas as pd
import numpy as np

from tqdm import tqdm, tqdm_notebook
    
import numpy as np

# Модели
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_log_error, mean_squared_error

%matplotlib inline

## Загрузка данных

In [2]:
movies = pd.read_csv('../data/ml-latest-small/movies.csv', )
links = pd.read_csv('../data/ml-latest-small/links.csv')
tags = pd.read_csv('../data/ml-latest-small/tags.csv')
ratings = pd.read_csv('../data/ml-latest-small/ratings.csv',)

## Построение гибридной рекомендательной системы

 - Холодный старт (до 5 оценок) - рекомендуем наиболее популярные фильмы
 - Теплый старт (от 5 до 10 оценок) - рекомендуем фильмы на основе сожержания со стекингом
 - Горячий старт (от 10 до бесконечности оценок) - коллаборативная фильтрация с блендингом (Item, User based)

##### Холодный старт - наиболее популярные фильмы

In [3]:
# Рекомендательная система для холодного старта
movies_ratings = ratings.groupby('movieId').agg({'userId': np.count_nonzero, 
                                'rating': [np.median, np.var, np.average]})
movies_ratings.columns=['userid_count', 'movie_rating_median', 'movie_rating_var', 'movie_rating_average']
movies_ratings.head()

,userid_count,movie_rating_median,movie_rating_var,movie_rating_average
movieId,,,,
1,215,4.0,0.696990,3.920930
2,110,3.5,0.777419,3.431818
3,52,3.0,1.112651,3.259615
4,7,3.0,0.726190,2.357143
5,49,3.0,0.822917,3.071429


In [4]:
# Нормируем количество оценок пользователей и рейтинг фильмов
movies_ratings['norm_movie_raiting'] = movies_ratings.apply(lambda row: (row['movie_rating_average'])
                             /(movies_ratings['movie_rating_average'].max() 
                                   - movies_ratings['movie_rating_average'].min())
                 , axis=1)
movies_ratings['norm_userid_count'] = movies_ratings.apply(lambda row: (row['userid_count'])
                             /(movies_ratings['userid_count'].max() - movies_ratings['userid_count'].min())
                 , axis=1)
# Популярность фильма = нормированный райтинг * нормированное количество оценок
movies_ratings['popularity'] = movies_ratings['norm_userid_count'] * movies_ratings['norm_movie_raiting']
movies_popularity = movies_ratings.merge(movies, on='movieId', how='left', sort=False)[['title', 'genres', 'popularity']]

In [5]:
def cold_start(userId):
    movies = movies_popularity.sort_values('popularity', ascending=False)[['movieId', 'title', 'popularity']].head(10)
    return movies

#### Теплый старт - рекомендация на основе содержания

###### Сформируем следующие признаки, для того чтобы сделать Content-bases рекомендации:
    1) TF-IDF метрика на жанрах и тегах
    2) Средняя оценка пользователя и фильма
    3) Медианная оценка пользователя и фильма
    4) Дисперсия оценки пользователя и фильма
    5) Количество оценок пользователя и фильма
    
Так же обогатим модель результатами **логистической регрессии** для оценки пользователя, 
после обучим алгоритм **KNN** на полученных данных    

In [6]:
# Средняя оценка, медианное значение, дисперсия, количество оценок пользователей
user_ratings = ratings.groupby('userId').agg({'movieId': np.count_nonzero, 
                                'rating': [np.median, np.var, np.average]}).head()
user_ratings.columns=['movieid_count', 'user_rating_median', 'user_rating_var', 'user_rating_average']
user_ratings.head()

,movieid_count,user_rating_median,user_rating_var,user_rating_average
userId,,,,
1,232,5.0,0.640077,4.366379
2,29,4.0,0.649015,3.948276
3,39,0.5,4.370783,2.435897
4,216,4.0,1.727132,3.555556
5,44,4.0,0.980973,3.636364


In [7]:
# Сгруппируем тэги для фильмов
grouped_tags = tags.groupby('movieId').agg({'tag': [(lambda x: "|".join(x)), np.count_nonzero]})
grouped_tags.columns=['all_tags', 'all_tags_count']

# Сгруппируем тэги для фильмов по пользователями
grouped_user_tags = tags.groupby(['movieId', 'userId'])\
                        .agg({'tag': [(lambda x: "|".join(x)), np.count_nonzero]})
grouped_user_tags.columns=['user_tags', 'user_tags_count']

In [8]:
# Датасет фильмов и оценок пользователей к ним
movies_ratings_tags = movies.merge(ratings, on='movieId', how='left', sort=False)\
                       .merge(grouped_tags, on='movieId', how='left', sort=False)\
                       .merge(movies_ratings, on='movieId', how='left', sort=False)\
                       .merge(user_ratings, on='userId', how='left', sort=False)\
                       .merge(grouped_user_tags, on=['movieId', 'userId'], how='left', sort=False)

# Посмотрим на получившийся датасет
movies_ratings_tags['all_tags'] = movies_ratings_tags['all_tags'].fillna('')
movies_ratings_tags['user_tags'] = movies_ratings_tags['user_tags'].fillna('')
movies_ratings_tags = movies_ratings_tags.fillna(0)
movies_ratings_tags.head(5)

,movieId,title,genres,userId,rating,timestamp,all_tags,all_tags_count,userid_count,movie_rating_median,...,movie_rating_average,norm_movie_raiting,norm_userid_count,popularity,movieid_count,user_rating_median,user_rating_var,user_rating_average,user_tags,user_tags_count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08,pixar|pixar|fun,3.0,215.0,4.0,...,3.92093,0.871318,0.655488,0.571138,232.0,5.0,0.640077,4.366379,,0.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08,pixar|pixar|fun,3.0,215.0,4.0,...,3.92093,0.871318,0.655488,0.571138,44.0,4.0,0.980973,3.636364,,0.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09,pixar|pixar|fun,3.0,215.0,4.0,...,3.92093,0.871318,0.655488,0.571138,0.0,0.0,0.000000,0.000000,,0.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09,pixar|pixar|fun,3.0,215.0,4.0,...,3.92093,0.871318,0.655488,0.571138,0.0,0.0,0.000000,0.000000,,0.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09,pixar|pixar|fun,3.0,215.0,4.0,...,3.92093,0.871318,0.655488,0.571138,0.0,0.0,0.000000,0.000000,,0.0


In [9]:
# Функция для TF-IDF метрики
def tf_idf(row, value, dictionary):
    return (1/len(row.split('|')))*dictionary[value] if value in row else 0

In [10]:
# Сформируем список жанров:
genres_list = []
for i in movies.genres.str.split('|'):
    for j in i:
        genres_list.append(j)
        
### Итоговый словарь жанров:
genres_dict = {i:np.log(len(movies)/genres_list.count(i)) for i in genres_list}

In [11]:
# Добавим новые фичи в датасет (TF-IDF на жанрах):
for i in tqdm(genres_dict):
    movies_ratings_tags['tf_idf_'+i] = movies_ratings_tags.apply(lambda row: tf_idf(row['genres'], i, genres_dict), axis=1)

100%|██████████| 20/20 [00:25<00:00,  1.26s/it]


In [12]:
# Сформируем список тэгов
tags_list = []
for i in grouped_tags.all_tags.str.split('|'):
    for j in i :
        tags_list.append(j)
        
# Итоговый словарь тэгов
tags_dict = {i:np.log(len(movies)/tags_list.count(i)) for i in tags_list if tags_list.count(i)>10 and i!=''}

In [13]:
# Добавим новые фичи в датасет (TF-IDF на тэгах для пользователя и фильма):
for i in tqdm(tags_dict):
    movies_ratings_tags['tf_idf_'+i] = movies_ratings_tags\
                    .apply(lambda row: tf_idf(row['all_tags'], i, tags_dict), axis=1)
    movies_ratings_tags['user_tf_idf_'+i] = movies_ratings_tags\
                    .apply(lambda row: tf_idf(row['user_tags'], i, tags_dict), axis=1)

100%|██████████| 46/46 [02:37<00:00,  3.41s/it]


In [14]:
X = movies_ratings_tags.drop(['timestamp', 'movieId', 'title', 'genres', 'userId', 
                              'rating', 'all_tags', 'user_tags'], axis=1)
y = movies_ratings_tags['rating']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=4)

## Обучение (регрессия)

In [34]:
?Ridge

In [59]:
model = Ridge(alpha=0.0001, solver='saga')
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.3010366681761203

In [62]:
model = Lasso(alpha=0.0001)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.30235808880304527

In [60]:
predictions = model.predict(X_test)
mean_squared_error(predictions, y_test)

0.7591416609479026

#### Горячий старт - коллаборативная фильтрация